In [2]:
import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI
import pyodbc
import os

load_dotenv('.env')

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
driver = os.getenv('DRIVER')

connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=Yes"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [17]:
query = '''
SELECT 
    oe.TipoObjeto AS TipoOjeto
    , oe.IdentificadorEcm AS IdentificadorModulo
    , oe.Descricao
	, ove.IdentificadorEcm
	, ove.Codigo
	, ove.Descricao
FROM EngineCalculo.ObjetoEcm AS oe
INNER JOIN EngineCalculo.ObjetoVariavelEcm AS ove 
	ON ove.ObjetoId = oe.Id 
WHERE ove.Ativo = 1
ORDER BY oe.TipoObjeto
		, oe.IdentificadorEcm 
		, ove.IdentificadorEcm
                '''
resultado = cursor.execute(query)
registro = resultado.fetchall()


In [23]:
colunas = [column[0] for column in cursor.description]
data = [dict(zip(colunas, row)) for row in registro]
df = pd.DataFrame(data)
teste = df[df['Descricao'].str.contains('Capa')]


In [25]:
teste.Descricao.unique()

array(['Capacitância - Fase A - AT', 'Capacitância - Fase B - AT',
       'Capacitância - Fase C - AT',
       'Tendência de evolução da Capacitância - Fase A - AT',
       'Tendência de evolução da Capacitância - Fase B - AT',
       'Tendência de evolução da Capacitância - Fase C - AT',
       'Alerta para Capacitância Alta - Fase A - AT',
       'Alerta para Capacitância Alta - Fase B - AT',
       'Alerta para Capacitância Alta - Fase C - AT',
       'Alerta para Capacitância Muito Alta - Fase A - AT',
       'Alerta para Capacitância Muito Alta - Fase B - AT',
       'Alerta para Capacitância Muito Alta - Fase C - AT',
       'Alerta para tendência da Capacitância alta - Fase A - AT',
       'Alerta para tendência da Capacitância alta - Fase B - AT',
       'Alerta para tendência da Capacitância alta - Fase C - AT',
       'Capacitância inicial fase A - AT',
       'Capacitância inicial fase B - AT',
       'Capacitância inicial fase C - AT',
       'Capacitância inicial Fase R (A

In [18]:
df.EquipamentoSigmaId.values[0]
from src.ecm.ECM import ECM
import datetime


ecm = ECM()
data = ecm.request_results(datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'), datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'), df.EquipamentoSigmaId.values[0])

TypeError: Object of type int64 is not JSON serializable